In [1]:
import numpy as np
import pandas as pd
from datascience import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [2]:
cleaning, cleaned = open('haberman.data.txt').read().split("\n"), Table(['Age', 'Op Year', 'Num of Pos Aux Nodes', 'Survival Status'])
cleaning = cleaning[0:len(cleaning)-1]
for i in range(len(cleaning)):
    cleaned = cleaned.with_row([int(j) for j in cleaning[i].split(',')])
cleaned

Age,Op Year,Num of Pos Aux Nodes,Survival Status
30,64,1,1
30,62,3,1
30,65,0,1
31,59,2,1
31,65,4,1
33,58,10,1
33,60,0,1
34,59,0,2
34,66,9,2
34,58,30,1


In [3]:
# rows = open('haberman.data.txt').read().split("\n")
# test = Table(['1','2','3','4'])
# test.with_row([int(i) for i in rows[0].split(',')])

# cleaned = cleaned.with_row([int(j) for j in cleaning[0].split(',')])
# type(cleaned.column(0).item(0))
# cleaning[0].split(',')

# adult_data = open('adult.data.txt').read()

# cleaner = []
# for i in range(len(cleaning)):
#     cleaner.append(cleaning[i].split(", "))
# cleaned = Table(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'])
# for i in range(10000):
#     cleaned = cleaned.with_row(cleaner[i])
# cleaned

# adult_data = open('adult.data.txt').read()
# cleaned_data = adult_data.split('\n')
# splitted = make_array()
# for i in range(1000):
#     chunk = cleaned_data[i].split(', ')
# #     splitted += chunk
#     print(chunk)
#     splitted = np.append(splitted, list(chunk))
#     print(splitted)
# # cleaned_data[0].split(', ')
# # splitted = cleaned_data[0:15]
# # splitted1 = cleaned_data[15:30]
# # splitted, splitted1



# splitted = []

# def split(data):
#     if len(data) <= 15:
#         return data
#     else:
#         one_row = data[0:15]
# #       np.append(splitted, one_row)
#         splitted.append(one_row)
#         return split(data[15:])

# split(short_data)
# # test = Table(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'])
# # # for row in splitted:
# # #     print(row)
# # test.with_row(make_array(np.arange(16)))
# # test
# table = Table(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'])
# for i in range(len(splitted)):
#     table.with_row(splitted[i])
# splitted

# # tiles = Table(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'])
# # tiles.with_row(splitted[0])
# # tiles.with_rows(make_array(make_array('c', 2, 3), make_array('d', 4, 2)))

# # test.with_rows([np.arange(17)])

# # adult_data = pd.read_csv('adult.data.txt')
# # adult_data = open('adult.data.txt')
# # adult_data = open('adult.data.txt').read()
# # cleaned_data = adult_data.split(', ')
# # Table.read_table(cleaned_data)
# # require(adult_data)
# # setDT(adult_data)